簡単なレコメンデーションの実装を通して、レコメンドの仕組みを理解する。


【問題1】アイテムベース

In [13]:
import pandas as pd
import numpy as np

In [6]:
tour_dict = {"Hawaii":[1,1,3,3,3,3], 
             "Guam":[2,1,5,4,3,3], 
             "London":[4,2,2,2,0,1],
             "NewYork":[3,4,2,3,4,4]}

In [16]:
df = pd.DataFrame(tour_dict, dtype=np.int32)

In [17]:
df

,Hawaii,Guam,London,NewYork
0,1,2,4,3
1,1,1,2,4
2,3,5,2,2
3,3,4,2,3
4,3,3,0,4
5,3,3,1,4


In [30]:
# cos_similarity
def cos_similarity(vector1, vector2):
    vector1_norm = np.linalg.norm(vector1)
    vector2_norm = np.linalg.norm(vector2)
    cos_sim = np.dot(vector1, vector2) / (vector1_norm * vector2_norm) # 今回0vectorは考慮しない
    return cos_sim

In [31]:
# 各columnをvectorとして取り出す
hawii_vector = df['Hawaii'].values
guam_vector = df['Guam'].values
london_vector = df['London'].values
new_york_vector = df['NewYork'].values

In [32]:
vector_lists = [hawii_vector, guam_vector, london_vector, new_york_vector]

In [33]:
# 総当たりで類似度を算出する 4つ毎に括る
cos_similarity_list = []
for i in vector_lists:
    cos_sim_list = [] # reset用
    for j in vector_lists:
        cos_sim = cos_similarity(i, j)
        cos_sim_list += [cos_sim]
    cos_similarity_list += [cos_sim_list]
        
cos_similarity_list

[[1.0000000000000002,
  0.9733285267845753,
  0.6325989947852169,
  0.8919017444789036],
 [0.9733285267845753, 1.0, 0.719569435436076, 0.8366600265340756],
 [0.6325989947852169,
  0.719569435436076,
  1.0000000000000002,
  0.754624494751408],
 [0.8919017444789036, 0.8366600265340756, 0.754624494751408, 1.0]]

In [45]:
labels = df.columns.values

In [46]:
cos_df = pd.DataFrame(cos_similarity_list, index=labels, columns=labels)

In [47]:
cos_df

,Hawaii,Guam,London,NewYork
Hawaii,1.000000,0.973329,0.632599,0.891902
Guam,0.973329,1.000000,0.719569,0.836660
London,0.632599,0.719569,1.000000,0.754624
NewYork,0.891902,0.836660,0.754624,1.000000


In [51]:
user_array = np.array([1,0,0,0])
user_array = user_array[np.newaxis,:]
user_array

array([[1, 0, 0, 0]])

In [52]:
cos_array = cos_df.values

In [55]:
# Matrix_multiply
matmul = np.dot(user_array, cos_array)
matmul

array([[1.        , 0.97332853, 0.63259899, 0.89190174]])

In [57]:
recomend_df = pd.DataFrame(matmul, columns=labels)
recomend_df

,Hawaii,Guam,London,NewYork
0,1.0,0.973329,0.632599,0.891902


当然だがハワイが一番高い.ハワイを除くならGuamが高いのでGuamをオススメできる

LondonとNewyorkに差がでる理由はわからない

【問題2】ユーザーベース

In [61]:
index_list = ["user1", "user2", "user3", "user4", "user5"]
columns_list = ["ゼロから作るDeepLearning", 
                "Python機械学習プログラミング", 
                "ゼロから作るDeepLearning-自然言語処理編-",
                "はじめてのパターン認識",
                "これからの強化学習"]

In [66]:
book_df = pd.DataFrame({"ゼロから作るDeepLearning":[2,1,0,2,1],
                        "Python機械学習プログラミング":[2,1,5,4,3],
                        "ゼロから作るDeepLearning-自然言語処理編-":[4,2,2,2,0],
                        "はじめてのパターン認識":[np.nan, 4,2,3,4],
                        "これからの強化学習":[np.nan, 3,5,4,1]},
                       index = index_list)

In [76]:
book_df

,ゼロから作るDeepLearning,Python機械学習プログラミング,ゼロから作るDeepLearning-自然言語処理編-,はじめてのパターン認識,これからの強化学習
user1,2,2,4,NaN,NaN
user2,1,1,2,4.0,3.0
user3,0,5,2,2.0,5.0
user4,2,4,2,3.0,4.0
user5,1,3,0,4.0,1.0


In [72]:
# user毎の相関を出す peason
user_corr_df = book_df.T.corr('pearson')

In [74]:
user_corr_df['user1'].sort_values

<bound method Series.sort_values of user1    1.000000
user2    1.000000
user3   -0.114708
user4   -0.500000
user5   -0.755929
Name: user1, dtype: float64>

user1に対してuser2の相関が高い.

In [84]:
book_df.loc['user2'].sort_values(ascending=False)

はじめてのパターン認識                    4.0
これからの強化学習                      3.0
ゼロから作るDeepLearning-自然言語処理編-    2.0
Python機械学習プログラミング              1.0
ゼロから作るDeepLearning             1.0
Name: user2, dtype: float64

user2の評価が最も高いのははじめてのパターン認識 4.0

であるので"はじめてのパターン認識”をリコメンドするとよいと考えられる